In [10]:
import numpy as np
from AbstractMesh import AbstractMesh
import utils
from scipy.sparse import *

In [21]:
class Quadmesh(AbstractMesh):
    def __init__(self, vertices= None, faces = None, face_normals = None, labels = None):
        
        self.face_normals    = None #npArray (Nx3)
        self.face_labels     = None #npArray (Nx1)
        self.faces           = None #npArray (Nx4)
        self.face2face       = None #npArray (Nx3?)
        
        if vertices and faces:
            self.vertices = np.array(vertices)
            self.faces = np.array(faces)
            
            if(faces_normals):
                self.face_normals = np.array(face_normals)
                
            if(labels):
                self.labels = np.array(labels)
                            
            super(Quadmesh, self).__init__()
            
    
    def show(self):
        pass
        
        
    def add_face(self, face_id0, face_id1, face_id2, face_id3):
        
        self.add_faces([face_id0, face_id1, face_id2, face_id3])
        
        
    def add_faces(self, new_faces):
        
        new_faces = np.array(new_faces)
        self.faces = np.concatenate([self.faces, new_faces])
        
        if new_faces[(new_faces[:,0] > self.num_vertices) | 
                     (new_faces[:,1] > self.num_vertices) | 
                     (new_faces[:,2] > self.num_vertices) | 
                     (new_faces[:,3] > self.num_vertices)].shape[0] > n_vert:
            raise Exception('The Id of a vertex must be lesser than the number of vertices')

        self.faces = np.concatenate([self.faces, new_faces])
        self.__compute_adjacencies()
        
        
    def remove_face(self, face_id):
        
        self.remove_faces([face_id])
        
        
    def remove_faces(self, face_ids):
        
        pass
        
    def removeVertex(self, vtx_id):
        
        self.vertices = np.delete(self.vertices, vtx_id)
        self.faces = np.faces[(self.faces[:,0] != vtx_id) &
                              (self.faces[:,1] != vtx_id) &
                              (self.faces[:,2] != vtx_id) &
                              (self.faces[:,3] != vtx_id)]
        
        self.faces[(self.faces[:,0] > vtx_id)][0] -=1
        self.faces[(self.faces[:,1] > vtx_id)][1] -=1
        self.faces[(self.faces[:,2] > vtx_id)][2] -=1
        self.faces[(self.faces[:,3] > vtx_id)][2] -=1
        
        self.__compute_adjacencies()
        
        
    def removeVertices(self, vtx_ids):
        raise NotImplementedError
        
        
    def load_from_file(self, filename):
        
        self.vertices, self.faces, self.face_normals = utils.read_obj(filename)
        self.__compute_adjacencies()
        
        
    def save_file(self, filename):
        raise NotImplementedError
    
    
    def __compute_adjacencies(self):
        
        map_ = dict()
        adjs =  [[] for i in range(self.faces[:].shape[0])]

        edges = np.c_[self.faces[:,0], self.faces[:,1], 
                      self.faces[:,1], self.faces[:,2], 
                      self.faces[:,2], self.faces[:,3], 
                      self.faces[:,3], self.faces[:,0]]
        edges.shape = (-1, 2)
        faces_idx = np.repeat(np.array(range(self.faces.shape[0])), 4)

        for e, f in zip(edges, faces_idx):
            e = (e[0], e[1])

            try:
                tmp = map_[e]
            except KeyError:
                tmp = None

            if tmp is None:
                map_[(e[1], e[0])] = f
            else:
                adjs[f].append(map_[e])
                adjs[map_[e]].append(f)

        self.face2face = np.array([a.append(-1) if len(a) < 4 else a for a in adjs])
    
        
    @property
    def export_triangles(self):
        
        tris = np.c_[self.faces[:,:3], self.faces[:,2:], self.faces[:,0]]
        tris.shape = (-1, 3)
        return tris
    
    @property
    def num_faces(self):
        
        return self.faces.shape[0]
    
    
    def simplex_centroids(self):
        
        if self._AbstractMesh__simplex_centroid is None:
            self._AbstractMesh__simplex_centroids = self.vertices[self.faces].mean(axis = 1)
            
        return self._AbstractMesh__simplex_centroid
    
    
    def __comput_metrics(self):
        
        self.simplex_metrics['aspect_ratio'] = aspect_ratio(self.vertices, self.faces)

In [22]:
qm = Quadmesh()

In [23]:
qm.load_from_file('bunny_quad.obj')

In [25]:
qm.num_faces

2412

In [64]:
qm.export_triangles

array([[   2,    1,    0],
       [   0,    3,    2],
       [   5,    4,    0],
       ...,
       [2408, 2409, 2411],
       [2413, 2412, 2410],
       [2410, 2411, 2413]])